# Criteo Click Through Rate Prediction 
Final project for W261 - Machine Learning at Scale  
By Ben Arnoldy, Kenneth Chen, Nick Conidas, Rohini Kashibatla, Pavan Kurapati

Criteo is a company that uses machine learning to serve up web advertising. In 2014, the company launched the [Click Through Rate (CTR) prediction competition](https://www.kaggle.com/c/criteo-display-ad-challenge) hosted on Kaggle. The group with the lowest loss score won. Our team is working on this challenge for our W261 class at UC Berkeley in order to showcase a semester of learning.

## Question 1: Question Formulation  
_Introduce the goal of your analysis. What questions will you seek to answer, why do people perform this kind of analysis on this kind of data? Preview what level of performance your model would need to achieve to be practically useful._

The overall question we are trying to answer is, given a set of information about a particular website visitor, session, ad, etc., will that visitor click the ad we serve?  
  
A company like Criteo could then use the answer to that question to decide which ad out of a series of possible ads to serve in a particular location during a particular session. Companies like Criteo are paid by advertisers at some rate per user click, so to maximize revenue, Criteo would serve the ad that maximizes (CTR * pay per click). If we can predict CTR accurately -- which is the goal of this competition -- Criteo can find the ad that will maximize their revenue.  
  
This has huge implications because online advertising is a giant industry.  
  
Spending on digital advertising globally is projected to be $327.28 billion in 2019, according to [eMarketer](https://www.emarketer.com/content/global-ad-spending-update).  
  
In order for this machine learning prediction to be useful, it must meet a few criteria:  
  
1. The prediction must happen in a split second. No one wants to wait for webpages to load but this prediction must happen at load time. So algorithm must ingest all the relevant feature data -- info about the user, the session, the ad, etc. -- plug it into the model and return a verdict, and then do that across all potential ads the company can serve, then pick the ad that will maximize revenue and serve it on the page. The pace required immediately rules out learning algorithms like K-Means where you can't pre-train a model. 
2. For companies like Criteo that are serving ads, it doesn't matter all that much _why_ certain ads have higher CTR; it's just crucial to know _which_ ads will. So, interpretability of the model isn't critical here, opening up the range of algorithms to things like neural nets and random forests. Criteo's customers, of course, would love to have an interpretable model so they could make more effective web ads, but that's a different use case.  
3. Web advertising companies are serving a lot of web ad impressions, and most of the time ads go unclicked. And there are a lot of possible features to track in the online space. So we need a lot of data to find something meaningful out of the high-dimensional and sparse data. Fortunately, we have it: the Criteo dataset for this competition has 45 million rows and represents just a week's worth of ad serves. Unfortunately, data on this scale is challenging to work with.
4. Whatever model we train will be quickly out of data given the rapid pace of changes in Internet behavior and the growing amount of data that can be used to improve models. That said, it's not clear that the model needs to be an online model where things are changing so fast that it must update instantly as more data streams in. The model training could happen offline in a matter of hours, even days, and still be okay. DO PEOPLE AGREE WITH THIS LAST POINT? I'M NOT SURE.


## Question 2: Algorithm Explanation
_Create your own toy example that matches the dataset provided and use this toy example to explain the math behind the algorithm that you will perform._  

Logistic regression carries over some of the same concepts as linear regression but with some important differences. Fundamentally, we are still training a model using the equation for a line, and the model is the set of coefficients or weights. 

\begin{equation}\tag{2.1}
y = \beta_0 + \beta_1 \cdot x_1 ... + \beta_m \cdot x_m
\end{equation}
  
\begin{equation}\tag{2.2}
w = [\beta_0, \beta_1, ... \beta_m]
\end{equation}  
  
where $m$ is number of features, $\beta_0$ is the intercept or bias term, and $w$ are the weights of our model.  
  
We aren't training on just one sample, of course, but over many, so the weights are actually calculated over many rows $x$'s and $y$'s.   
  
\begin{equation}\tag{2.3}
y_j = \displaystyle\sum_{i=1}^{m}{w_i\cdot x_{ji} + b}
\end{equation}  
  
where $b$ is the intercept of bias term.  

In vector notation, this can be written:  

\begin{equation}\tag{2.4}
\mathbf{y_j} = \displaystyle{\mathbf{w}^T\mathbf{x}_{j} + b}
\end{equation}  
  
And we can simplify further by including the bias term b into the weights and augmenting the $\mathbf{x}$ vector with a value of 1 that will pass through the bias term as b * 1:

\begin{equation}\tag{2.5}
\mathbf{x}' := 
\begin{bmatrix}
\mathbf{x}\\
1
\end{bmatrix},\quad
\boldsymbol{\theta} :=
\begin{bmatrix}
\mathbf{w}\\
b
\end{bmatrix}
\end{equation}  
  
leaving us with more compact vector equation for the line:  
  
\begin{equation}\tag{2.6}
h(\boldsymbol{\theta}) = \boldsymbol{\theta}^T\mathbf{x}'
\end{equation}
  
As with linear regression, we use this line to make predictions. In linear regression, we take a new datapoint that has a vector of x-values ($\mathbf{x}'$), plug it into the linear equation with our model coefficients ($\boldsymbol{\theta}$), and get a predicted value h(\boldsymbol{\theta}) that falls on the line.  
  
With logistic regression, however, that isn't our final answer. We aren't just trying to get some continuous value y coordinate for our x, we are trying to arrive at a binary classification of a 1 or a 0. 

To do that we must do two additional steps. First, we want to contrain our $h(\boldsymbol{\theta})$ value to the range of $[0,1]$ and do so in a way groups the data more heavily toward either 0 or 1. We do that through a sigmoid function, whose output can be charted like this ([chart source](https://ml-cheatsheet.readthedocs.io/en/latest/logistic_regression.html)):

In [ ]:
from IPython.display import Image
Image(filename="sigmoid.png", width="400", height="200")

The function for the sigmoid is as follows:  

\begin{equation}\tag{2.7}
P(class=1) = \frac{1} {1 + e^{-z}}
\end{equation}  

where $P(class=1)$ is the probability that our predicted value $z$ should be classified as 1. Again, $z$ is our predicted value, so it can thought of as  $h(\boldsymbol{\theta})$ or $\boldsymbol{\theta}^T\mathbf{x}'$. So we can write this sigmoid function another way:   
  
\begin{equation}\tag{2.8}
P(class=1) = \frac{1} {1 + e^{-\boldsymbol{\theta}^T\mathbf{x}'}}
\end{equation}
  
Equation 2.8 formed the basis for our sigmoid function that we wrote for our toy dataset in section 3 below.  
  
Once we have these probabilities, we need to pick a decision boundary between 0 and 1. One logical default value for that boundary is 0.5. So if the probability is greater than 0.5, classify the sample as 1, otherwise classify it as 0. In chart form, that would look like ([chart source](https://ml-cheatsheet.readthedocs.io/en/latest/logistic_regression.html)):  

In [ ]:
Image(filename="sigmoid_w_threshold.png", width="400", height="200")

And, in mathemathical form, it looks like:  
\begin{equation}\tag{2.9}
p \geq 0.5, class=1 \\
p < 0.5, class=0
\end{equation}

Equation 2.9 formed the basis for our decision_boundary function that we wrote in section 3 for our toy dataset.  
  
Like with linear regression, in order to parallelize the training of the linear model we can't do a closed-form solution but must treat it as an optimization problem with gradient descent. Fortunately, the Log Loss or Cross-Entropy Loss gives us a convex equation, allowing us to use gradient descent with the confidence of not hitting local minima.   
  
The Log Loss function is much like what it sounds: the negative of the log of the prediction value, though there is two versions depending on the actual label value.  
  
\begin{equation}\tag{2.10}
Cost(h_\theta(x),y) = - log(h_\theta(x)), if y=1 \\
Cost(h_\theta(x),y) = - log(1 - h_\theta(x)), if y=0
\end{equation}  
  
We can compile this into a single cost function taking advantage of the fact that we can zero out either side of the equation given $y$ being either 0 or 1 by multiplying one side of the equation by $y$ and another side by $(1-y)$:  

\begin{equation}\tag{2.11}
J(\theta) = - \frac{1}{m}\sum_{i=1}^{m}{[y_i log(h_\theta(x_i)) + (1 - y_i) log(1 - h_\theta(x_i))]}
\end{equation}  

Using equation 2.6 from above ($h(\boldsymbol{\theta}) = \boldsymbol{\theta}^T\mathbf{x}'$), this can be rewritten as:  

\begin{equation}\tag{2.12}
J(\theta) = \frac{1}{m}\cdot(-\mathbf{y}\cdot\boldsymbol{\theta}^T\mathbf{x}' - (1 - \mathbf{y})\cdot log(1 - \boldsymbol{\theta}^T\mathbf{x}')
\end{equation}  

Equation 2.12 forms the basis of our spark job in the LogLoss function that we built for the toy dataset in section 3 below. 

For the gradient, we take the first derivative of this cost function, which winds up being:  
  
\begin{equation}\tag{2.13}
\nabla_\theta f = \mathbf{x}'\cdot(h(\boldsymbol{\theta}) - \mathbf{y}) \\
\nabla_\theta f = \mathbf{x}'\cdot(\frac{1}{1+e^{-\boldsymbol{\theta} \mathbf{x'}}} - \mathbf{y}) \\
\nabla_\theta f = \frac{x}{1+e^{-\boldsymbol{\theta} \mathbf{x'}}} -\mathbf{x}'\mathbf{y} \\
\nabla_\theta f = (\frac{1}{1+e^{-\mathbf{y}\boldsymbol{\theta} \mathbf{x'}}} - 1)\cdot \mathbf{x}'\mathbf{y}
\end{equation}  
  
Equation 2.13 forms the basis of our spark job in the GDUpdate_wReg function that we built for the toy dataset in section 3 below.  

## Question 3: EDA...... code for RDDs below

In [121]:
import re
import ast
import time
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numbers

from pyspark.ml.feature import FeatureHasher
from pyspark.sql import SQLContext, Row
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import SparseVector
from pyspark.sql.types import IntegerType
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import FeatureHasher
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import OneHotEncoderEstimator

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]

In [5]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "fproj_notebook"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

#### Load the data

In [9]:
# load the data into Spark RDD 
projectRDD = sc.textFile('gs://w261-bucket-pav/notebooks/data/train.txt')

In [8]:
# helper function
def extractLabel(line):
    """Mapper to extract labels"""
    label = line[0]
    yield label

def extractTrain(line):
    """ Extracts train data"""
    train = line[1:]
    yield train

In [10]:
# Convert string row to individual features and label
projectRDD = projectRDD.map(lambda x: x.split('\t')).cache()

In [ ]:
# Split the RDD into train, val and test
trainRDD,valRDD,testRDD=projectRDD.randomSplit([0.8, 0.1, 0.1],2019)

In [14]:
print(f"... held out {valRDD.count()} records for validation, {testRDD.count()} records for test and assigned {trainRDD.count()} for training.")

... held out 4585453 records for validation, 4586499 records for test and assigned 36668665 for training.


In [24]:
def parse(element):
    """
    Map record_csv_string --> (tuple,of,fields)
    """
    n_elements = len(element)
    fields = np.array(element)
    features,click = fields[1:], fields[0]
    return(features, click)

In [25]:
trainRDDCached = trainRDD.map(parse).cache()

#### Extract a toy dataset
To just show the workings of the algorithm, we want a small number of rows and avoid types of data that we can't handle. That means we need to remove categorical features and NAs. We explored a number of possibilities for handling the NA problem:
1. Simply removing all rows with missing data. The problem: That results in the loss of approximately 90 percent of the data. 
2. Creating dummy variables for each integer feature that would be set to True if the feature had data or False if the feature had no data. Then, we would replace all NAs in the integer features with zeros. So, for I1 there would be an I1_missing, for I2 an I2_missing, and so on. We did not see an accuracy gain from those Ix_missing features and instead saw L1 Normalization zeroing those out. 
3. Replace the NAs with mean values of each feature. This seemed to strike the right balance between preserving data and not creating unnecessary features. 

In [197]:
def remove_categorical(x):
    features = x[0]
    label = x[1]
    # We are interested in C6,C9, C14, C17, C20, C22, C23
    # These are at index 18,21,26,29,32,34,35
    c_index_list = [18,21,26,29,32,34,35]
    new_arr = np.array(features[0:13],dtype=str)
    for index in c_index_list:
        new_arr = np.append(new_arr,features[index])
    yield((new_arr,label))

new_trainRDDCached = trainRDDCached.flatMap(remove_categorical).cache()

In [212]:
# Replace all blanks in new_trainRDDCached integer values with their means

def replace_int_null(x):
    features = x[0]
    label = int(x[1])
    for i in range(13):
        if features[i]=="":
            features[i] = float(mean_dict_bc.value[i+1])
        else:
            features[i] = float(features[i])
    features = features.astype(float)
    yield((features,label))

def replace_int_null_noInt(x):
    features = x[0]
    label = int(x[1])
    for i in range(13):
        if features[i]=="":
            features[i] = float(mean_dict_bc.value[i+1])
        else:
            features[i] = float(features[i])
    yield((features,label))
    
def replace_char_null(x):
    features = x[0]
    label = int(x[1])
    for i in range(13,len(features)):
        if features[i]=="":
            features[i] = 'AAAA'
    yield((features,label))
    

In [213]:
new_trainRDDCached = new_trainRDDCached.flatMap(replace_int_null_noInt)\
                                    .flatMap(replace_char_null).cache()

[(array(['2.0', '0.0', '44.0', '1.0', '102.0', '8.0', '2.0', '2.0', '4.0',
         '1.0', '1.0', '0.991518', '4.0', 'fe6b92e5', 'a73ee510',
         'b28479f6', '07c540c4', '5840adea', 'AAAA', '3a171ecb'],
        dtype='<U8'), 0)]

In [214]:
IntFeatRDD = IntFeatRDD.flatMap(replace_int_null).cache()

[(array([  2.      ,   0.      ,  44.      ,   1.      , 102.      ,
           8.      ,   2.      ,   2.      ,   4.      ,   1.      ,
           1.      ,   0.991518,   4.      ]), 0)]

#### Normalize the toy data set

In [217]:
def normalize(dataRDD):
    """
    Scale and center data round mean of each feature.
    Args:
        dataRDD - records are tuples of (features_array, y)
    Returns:
        normedRDD - records are tuples of (features_array, y)
    """
    featureMeans = dataRDD.map(lambda x: x[0]).mean()
    featureStdev = np.sqrt(dataRDD.map(lambda x: x[0]).variance())
    normedRDD = dataRDD.map(lambda x: (np.divide((x[0] - featureMeans), featureStdev), x[1]))        
    return normedRDD

In [218]:
normedRDD = normalize(IntFeatRDD).cache()

#### Run logistic regression, include L1, L2 normalization options

In [ ]:
def LogLoss(dataRDD, W):
    """
    Compute loss using log loss or cross entropy formula.
    Args:
        dataRDD - each record is a tuple of (features_array, y)
        W       - (array) model coefficients with bias at index 0
    Adapted from sources:
    * Homework 4 code for W261
    * https://ml-cheatsheet.readthedocs.io/en/latest/logistic_regression.html
    * https://www.internalpointers.com/post/cost-function-logistic-regression
    """
    augmentedData = dataRDD.map(lambda x: (np.append([1.0], x[0]), x[1]))

    loss = augmentedData.map(lambda x: (-np.dot(x[1],np.log(1 / (1 + np.exp(-np.dot(W, x[0]))))) -\
                                        np.dot((1-x[1]),np.log(1- (1 / (1 + np.exp(-np.dot(W, x[0])))))), 1)) \
                        .reduce(lambda x,y:(x[0]+y[0], x[1]+y[1]))

    loss = float(loss[0])/loss[1]
    
    return loss

In [ ]:
def GDUpdate_wReg(dataRDD, W, N, learningRate = 0.1, regType = None, regParam = 0.1):
    """
    Perform one gradient descent step/update with ridge or lasso regularization.
    Args:
        dataRDD - tuple of (features_array, y)
        W       - (array) model coefficients with bias at index 0
        learningRate - (float) defaults to 0.1
        regType - (str) 'ridge' or 'lasso', defaults to None
        regParam - (float) regularization term coefficient
    Returns:
        model   - (array) updated coefficients, bias still at index 0
    Adapted from sources:
    * Homework 4 code from W261
    * Notebook cited by Jimi in asynch: https://nbviewer.jupyter.org/urls/dl.dropbox.com/s/r20ff7q0yni5kiu/LogisticRegression-Spark-Notebook.ipynb
    """
    
    # augmented data
    # this puts a 1.0 in front of x's to pass thru bias term
    augmentedData = dataRDD.map(lambda x: (np.append([1.0], x[0]), x[1])) 
    
    new_model = None
    
    # calculate the gradient
    
    gradient = augmentedData.map(lambda x: (1 / (1 + np.exp(-x[1]*np.dot(W, x[0])))-1) * x[1] * np.array(x[0]))\
                    .reduce(lambda x, y: x + y)
    if regType == "ridge":
        wReg = W * 1
        wReg = np.append([0], wReg[1:]) # remove the bias term ahead of regularization        
    elif regType == "lasso":
        wReg = W * 1
        wReg = np.append([0], wReg[1:]) # remove the bias term ahead of regularization
        wReg = (wReg>0).astype(int) * 2-1
    else:
        wReg = np.zeros(W.shape[0])
    gradient = gradient + regParam * wReg  #gradient:  GD of Squared Error+ GD of regularized term 
    new_model = W - learningRate * gradient / N

    return new_model

In [ ]:
def GradientDescent_wReg(trainRDD, testRDD, wInit, nSteps = 20, learningRate = 0.1,
                         regType = None, regParam = 0.1, verbose = True):
    """
    Perform nSteps iterations of regularized gradient descent and 
    track loss on a test and train set. Return lists of
    test/train loss and the models themselves.
    Adapted from sources:
    * Homework 4 code from W261
    * Notebook cited by Jimi in asynch: https://nbviewer.jupyter.org/urls/dl.dropbox.com/s/r20ff7q0yni5kiu/LogisticRegression-Spark-Notebook.ipynb    
    """
    # initialize lists to track model performance
    train_history, test_history, model_history = [], [], []
    
    # calculate N here so you don't have to do it in the for looped function call
    N = trainRDD.count()

    # make a starter set of weights if none provided
    featureLen = len(trainRDD.take(1)[0][0])
    if wInit is None:
        w = np.random.normal(size=featureLen) # w should be broadcasted if it is large
    else:
        w = wInit
    model = w
    
    # perform nSteps updates & compute test and train loss after each
    for idx in range(nSteps):  
   
        # update the model
        model = GDUpdate_wReg(trainRDD, model, N, learningRate, regType, regParam)
        
        # keep track of test/train loss for plotting
        training_loss = LogLoss(trainRDD, model)
        test_loss = LogLoss(testRDD, model)
        train_history.append(training_loss)
        test_history.append(test_loss)
        model_history.append(model)
        
        # console output if desired
        if verbose:
            print("----------")
            print(f"STEP: {idx+1}")
            print(f"training loss: {training_loss}")
            print(f"test loss: {test_loss}")
            print(f"Model: {[round(w,3) for w in model]}")
    return train_history, test_history, model_history

In [ ]:
# split data into train and test and train a logistic regression model with L1 regularization

# initialize the weights with a bias term determined after some testing and zeros for coeffecients
wInit = np.array([-0.77, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

trainRDD, testRDD = normedNONARDD.randomSplit([0.8,0.2], seed = 5)
start = time.time()
lasso_results = GradientDescent_wReg(trainRDD, testRDD, wInit, nSteps = 10, regParam = 0.1)
print(f"\n... trained {len(lasso_results[2])} iterations in {time.time() - start} seconds")

In [ ]:
# make error curve plots that show declining loss as walk down the gradient
# from HW4 code

def plotErrorCurves(trainLoss, testLoss, title = None):
    """
    Helper function for plotting.
    Args: trainLoss (list of log loss) , testLoss (list of log loss)
    """
    fig, ax = plt.subplots(1,1,figsize = (16,8))
    x = list(range(len(trainLoss)))[1:]
    ax.plot(x, trainLoss[1:], 'k--', label='Training Loss')
    ax.plot(x, testLoss[1:], 'r--', label='Test Loss')
    ax.legend(loc='upper right', fontsize='x-large')
    plt.xlabel('Number of Iterations')
    plt.ylabel('Log Loss')
    if title:
        plt.title(title)
    plt.show()

In [ ]:
# save results to csv files
# from HW4 code

trainLoss, testLoss, models = lasso_results
np.savetxt(PWD + '/data/lasso_models.csv', np.array(models), delimiter=',')
np.savetxt(PWD + '/data/lasso_loss.csv', np.array([trainLoss, testLoss]), delimiter=',')
plotErrorCurves(trainLoss, testLoss, title = 'Lasso Regression Error Curves' )

In [ ]:
# get the best model from files
# from HW4 code

saved_models = np.loadtxt(PWD + '/data/lasso_models.csv', dtype=float, delimiter=',')
best_model = saved_models[-1,:]

In [ ]:
# show the coeffecients as L1 steps increase
# adapted from HW4 code

def plotCoeffs(models, featureNames, title):
    """
    Helper Function to show how coefficients change as we train.
    """
    fig, ax = plt.subplots(figsize = (15,8))
    X = list(range(len(models)))
    for data, name in zip(models.T, featureNames):
        if name == "Bias":
            continue
        ax.plot(X, data, label=name)
    ax.plot(X,[0]*len(X), 'k--')
    plt.title(title)
    plt.legend()
    plt.show
    
#use if starting with the models with the missing data features
#plotCoeffs(saved_models, ['Bias'] + numeric_features + missing, "Lasso Coefficients over 50 GD steps")

# use if starting with the model where NA rows were dropped
plotCoeffs(saved_models, ['Bias'] + numeric_features, "Lasso Coefficients over 50 GD steps")

In [ ]:
# test accuracy of the model
# from learning here: https://ml-cheatsheet.readthedocs.io/en/latest/logistic_regression.html

def sigmoid(row):
    sig = []
    for z in row:
        sig.append(1/(1+np.exp(-z)))
    return sig

def decision_boundary(prob):
    return 1 if prob >= .5 else 0

def classify(predictions):
    '''
    input  - N element array of predictions between 0 and 1
    output - N element array of 0s (False) and 1s (True)
    '''
    classifications = []
    for prediction in predictions:
        classifications.append(decision_boundary(prediction))
    return classifications

def calc_accuracy(predicted_labels, actual_labels):
    error_count = 0
    num_predictions = len(actual_labels)
    for compare in range(num_predictions):
        if predicted_labels[compare] != actual_labels[compare]:
            error_count += 1
    return 1.0 - (float(error_count) / num_predictions)

def compare(x,y):
    return int(x != y)

coeffs = best_lasso[1:] # coefficients
b = best_lasso[0] # bias term
actual_labels = testRDD.map(lambda x: x[1]).collect()

# regular version -- good for toy example, may need spark below on big data
mxb = testRDD.map(lambda x: np.dot(coeffs, x[0])+b).collect() # mx+b
probabilities = sigmoid(mxb) # put through sigmoid
predictions = classify(probabilities) # put through a decision boundary
print ("accuracy is", calc_accuracy(predictions, actual_labels))

# spark version below if regular version crashes with big data
"""
error_count = testRDD.map(lambda x: (np.dot(coeffs, x[0])+b, x[1])) \
                    .map(lambda x: (1/(1+np.exp(-x[0])), x[1])) \
                    .map(lambda x: (decision_boundary(x[0]), x[1])) \
                    .map(lambda x: (compare(x[0],x[1]),1)) \
                    .reduce(lambda x,y: (x[0]+y[0],x[1]+y[1]))

print("accuracy is", 1.0 - (float(error_count[0]) / error_count[1]))
"""